In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType,BooleanType

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/27 06:22:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df_product = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')
df_comment = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Comment')
df_shop = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/ShopInfo')

In [4]:
df_product.createOrReplaceTempView("product")
df_comment.createOrReplaceTempView("comment")
df_shop.createOrReplaceTempView("shop")

In [5]:
print("comment count: {} \nproduct count: {} \nshop count: {}".format(df_comment.count(),df_product.count(),
                                       df_shop.count()))

comment count: 4243515 
product count: 121040 
shop count: 9945


In [6]:
df_product

DataFrame[id: int, master_id: int, sku: string, price: int, list_price: int, original_price: int, discount: int, discount_rate: float, rating_average: float, review_count: int, productset_group_name: string, all_time_quantity_sold: int, name: string, short_description: string, clean_name: string, clean_description: string, clean_specifications: string, category_name: string, seller_id: string, seller_name: string, seller_store_id: string, spid: int, category_id: int]

In [7]:
df_product_shop = spark.sql("""
    select p.id p_id, p.master_id,p.price,p.list_price,p.original_price,p.discount,p.discount_rate,
    p.productset_group_name,p.all_time_quantity_sold,p.name p_name,p.short_description,p.clean_name p_clean_name,
    p.clean_description,p.clean_specifications,
    p.category_name,p.category_id,
    p.seller_id, s.store_id,s.name s_name,s.icon,s.url,s.is_official,s.store_level,s.is_followed,
    s.avg_rating_point,s.review_count, s.total_follower,s.days_since_joined
    from product p
    join shop s on p.seller_id = s.id
""")

In [8]:
df_product_shop.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Product_Shop')

22/11/27 06:23:57 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
df_all = spark.sql("""
    select p.id p_id, p.master_id,p.price,p.list_price,p.original_price,p.discount,p.discount_rate,
    p.productset_group_name,p.all_time_quantity_sold,p.name p_name,p.short_description,p.clean_name p_clean_name,
    p.clean_description,p.clean_specifications,
    p.category_name,p.category_id,
    p.seller_id, s.store_id,s.name s_name,s.icon,s.url,s.is_official,s.store_level,s.is_followed,
    s.avg_rating_point,s.review_count, s.total_follower,s.days_since_joined, 
    c.id c_id,c.content, c.clean_content,c.thank_count,c.customer_id,c.rating,
    c.customer_full_name,c.purchased_at,c.date_purchased_at,c.review_created_date,
    c.delivery_date,c.review_after_delivery,c.sentiment,c.lable
    from product p
    join shop s on p.seller_id = s.id
    join comment c on c.product_id = p.id and p.seller_id = c.seller_id
""")

In [10]:
df_all.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/metaData')

In [11]:
spark.stop()